<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/alzheimer_mri_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install opendatasets library
!pip install opendatasets

In [ ]:
# Install gradio library
!pip install gradio==3.14.0

In [ ]:
# Import necessary libraries and functions
from tensorflow.keras.layers import MaxPooling2D,Dense,Flatten,Conv2D,Input,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import opendatasets as od
from keras.callbacks import ModelCheckpoint
import os
import shutil
import gradio as gr
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
import matplotlib.pyplot as plt

In [ ]:
# Download the dataset from kaggle
od.download("https://www.kaggle.com/datasets/sachinkumar413/alzheimer-mri-dataset")

In [7]:
SEED=42#randomseed=42

In [3]:
Mild_Demented_path="/content/alzheimer-mri-dataset/Dataset/Mild_Demented"
Moderate_Demented_path="/content/alzheimer-mri-dataset/Dataset/Moderate_Demented"
Very_Mild_Demented_path="/content/alzheimer-mri-dataset/Dataset/Very_Mild_Demented"
# Create Demented directory
Demented_path=os.path.join("/content/alzheimer-mri-dataset/Dataset","Demented")
os.makedirs(Demented_path,exist_ok=True)
# Set the list for each class
mild_demented_images=list(os.listdir(Mild_Demented_path))
moderate_demented_images=list(os.listdir(Moderate_Demented_path))
very_demented_images=list(os.listdir(Very_Mild_Demented_path))

In [4]:
for image in mild_demented_images:
  source_path=os.path.join(Mild_Demented_path,image)
  destination_path=os.path.join(Demented_path,image)
  shutil.move(source_path,destination_path)
for image in moderate_demented_images:
  source_path=os.path.join(Moderate_Demented_path,image)
  destination_path=os.path.join(Demented_path,image)
  shutil.move(source_path,destination_path)
for image in very_demented_images:
  source_path=os.path.join(Very_Mild_Demented_path,image)
  destination_path=os.path.join(Demented_path,image)
  shutil.move(source_path,destination_path)

In [8]:
classes=['Demented','Non_Demented']
for class_name in classes:
 train_dir=os.path.join("/content/alzheimer-mri-dataset/Dataset","train",class_name)
 os.makedirs(train_dir,exist_ok=True)
 valid_dir=os.path.join("/content/alzheimer-mri-dataset/Dataset","valid",class_name)
 os.makedirs(valid_dir,exist_ok=True)
from sklearn.model_selection import train_test_split
demented_train_images,demented_valid_images=train_test_split(
    list(os.listdir("/content/alzheimer-mri-dataset/Dataset/Demented")),test_size=0.2)
non_demented_train_images,non_demented_valid_images=train_test_split(
    list(os.listdir("/content/alzheimer-mri-dataset/Dataset/Non_Demented")),test_size=0.2)
for image,img in zip(demented_train_images,demented_valid_images):
  source_path_1=os.path.join("/content/alzheimer-mri-dataset/Dataset/Demented",image)
  destination_path_1=os.path.join("/content/alzheimer-mri-dataset/Dataset/train/Demented",image)
  shutil.move(source_path_1,destination_path_1)
  source_path_2=os.path.join("/content/alzheimer-mri-dataset/Dataset/Demented",img)
  destination_path_2=os.path.join("/content/alzheimer-mri-dataset/Dataset/valid/Demented",img)
  shutil.move(source_path_2,destination_path_2)


In [9]:
for image,img in zip(non_demented_train_images,non_demented_valid_images):
  source_path_1=os.path.join("/content/alzheimer-mri-dataset/Dataset/Non_Demented",image)
  destination_path_1=os.path.join("/content/alzheimer-mri-dataset/Dataset/train/Non_Demented",image)
  shutil.move(source_path_1,destination_path_1)
  source_path_2=os.path.join("/content/alzheimer-mri-dataset/Dataset/Non_Demented",img)
  destination_path_2=os.path.join("/content/alzheimer-mri-dataset/Dataset/valid/Non_Demented",img)
  shutil.move(source_path_2,destination_path_2)

In [ ]:
# Define the parameters for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=15,  # Rotate images randomly by up to 15 degrees
    width_shift_range=0.1,  # Shift images horizontally by up to 10% of the width
    height_shift_range=0.1,  # Shift images vertically by up to 10% of the height
    shear_range=0.1,  # Shear transformations with a maximum shear intensity of 10%
    zoom_range=0.1,  # Zoom in randomly by up to 10%
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True,  # Flip images vertically
    fill_mode='nearest'  # Fill missing pixels using the nearest value
)

# Define the directory containing your image data
train_dir = "/content/alzheimer-mri-dataset/Dataset/train"
valid_dir= "/content/alzheimer-mri-dataset/Dataset/valid"

# Create an ImageDataGenerator for the training data
train_generator =datagen.flow_from_directory(
    train_dir,
    batch_size=32,
    class_mode='binary'
)

valid_generator=datagen.flow_from_directory(
    valid_dir,
    batch_size=32,
    class_mode='binary'
)


In [ ]:
# Define input shape
input_shape = (256, 256, 3)  #  Input images are 256x256 RGB images

# Define input layer
inputs = Input(shape=input_shape)

# Convolutional layers
layer_1= Conv2D(12, kernel_size=(3, 3), activation='relu')(inputs)
layer_2 =MaxPooling2D(pool_size=(2, 2))(layer_1)
layer_3 = Conv2D(24, kernel_size=(3, 3), activation='relu')(layer_2)
layer_4 = MaxPooling2D(pool_size=(2, 2))(layer_3)

# Flatten layer
layer_5= Flatten()(layer_4)

# Dense layers
layer_6 = Dense(100, activation='relu')(layer_5)
outputs = Dense(1, activation='sigmoid')(layer_6)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision()])

# Print model summary
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 254, 254, 12)      336       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 12)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 24)      2616      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 62, 62, 24)        0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 92256)             0   

In [ ]:
# Define a checkpoint callback to save the best
checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
# Train your model with the callback
history=model.fit(train_generator,validation_data=(valid_generator),epochs=30,batch_size=len(train_generator),callbacks=[checkpoint])

In [ ]:
# Load the best weights into your model
model.load_weights('best_weights.h5')

In [ ]:
# Evaluate the model on valid_generator
model.evaluate(valid_generator)

40/40 [==============================] - 22s 535ms/step - loss: 0.5786 - accuracy: 0.7039 - precision_3: 0.7310


[0.5786424279212952, 0.703906238079071, 0.7309734225273132]

In [ ]:
# `history.history` is a dictionary containing the training and validation metrics
# Plotting accuracy
plt.subplot(2, 1, 1)  # 2 rows, 1 column, plot 1 (top subplot)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plotting precision
plt.subplot(2, 1, 2)  # 2 rows, 1 column, plot 2 (bottom subplot)
plt.plot(history.history['precision_3'])
plt.plot(history.history['val_precision_3'])
plt.title('Model Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()  # Adjust subplot layout to avoid overlap

# Show plot
plt.show()

In [ ]:
# Save the model as an HDF5 file
model.save('alzheimer_mri_model.h5')

In [ ]:
# Load MobileNetV1 base model without top (fully connected) layers
#base_model = MobileNet(weights='imagenet', include_top=False)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# Create a Sequential model
model = Sequential()

# Add the MobileNetV1 base model to the Sequential model
model.add(base_model)
# Set MobileNetV1 layers as non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
model.add(GlobalAveragePooling2D())

# Add custom dense layers for classification
model.add(Dense(220, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision()])
# Define a checkpoint callback to save the best
checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
# Train the model
history=model.fit(train_generator,validation_data=(valid_generator),epochs=15,batch_size=len(train_generator),callbacks=[checkpoint])

In [ ]:
# `history.history` is a dictionary containing the training and validation metrics
# Plotting accuracy
plt.subplot(2, 1, 1)  # 2 rows, 1 column, plot 1 (top subplot)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plotting precision
plt.subplot(2, 1, 2)  # 2 rows, 1 column, plot 2 (bottom subplot)
plt.plot(history.history['precision_3'])
plt.plot(history.history['val_precision_3'])
plt.title('Model Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()  # Adjust subplot layout to avoid overlap

# Show plot
plt.show()

In [ ]:
# Load the best weights into your model(second model has better performance over unseen data)
model.load_weights('best_weights.h5')

In [ ]:
# Save the second model
model.save("alzheimer_model_2.h5")

In [4]:
# Load the second model
model=load_model("/content/alzheimer_model_2.h5")

In [ ]:
# Evaluate the second model on valid_generator
model.evaluate(valid_generator)

40/40 [==============================] - 91s 2s/step - loss: 0.5019 - accuracy: 0.7516 - precision_1: 0.7738


[0.5018686652183533, 0.7515624761581421, 0.773809552192688]

In [5]:
def diagnosis(img_path):
    # Convert the numpy array to a PIL Image object
    img = Image.open(img_path)

    # Convert grayscale image to RGB
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Resize the image to match the model's expected input size
    img = img.resize((224, 224))

    # Normalize the image data
    img_array = np.array(img) / 255.0

    # Expand the dimensions to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Use the model to predict
    prediction = model.predict(img_array)
    #return prediction
    # Check if prediction is positive or negative
    if prediction <= 0.5:
        return 'Tumor presence: positive.'
    else:
        return 'Tumor presence: negative.'



In [32]:
# Example usage 1
img="/content/alzheimer-mri-dataset/Dataset/train/Demented/mild_106.jpg"
diagnosis(img)

1/1 [==============================] - 0s 66ms/step


'Tumor presence: positive.'

In [34]:
# Example usage 2
img="/content/alzheimer-mri-dataset/Dataset/train/Non_Demented/non_10.jpg"
diagnosis(img)

1/1 [==============================] - 0s 54ms/step


'Tumor presence: negative.'

In [7]:
# Define a function suitable for gradio
def gradio_fn(img_array):
    # Convert the numpy array to a PIL Image object
    img = Image.fromarray(np.uint8(img_array))

    # Convert grayscale image to RGB
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Resize the image to match the model's expected input size
    img = img.resize((224, 224))

    # Normalize the image data
    img_array = np.array(img) / 255.0

    # Expand the dimensions to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Use the model to predict
    prediction = model.predict(img_array)

    # Check if prediction is positive or negative
    if prediction <= 0.5:
        return 'Tumor presence: positive.'
    else:
        return 'Tumor presence: negative.'

In [ ]:
# Define the input component for the Gradio interface
inputs = gr.inputs.Image(shape=(224, 224))

# Define the output component for the Gradio interface
outputs = gr.outputs.Label()

# Create the Gradio interface
gr.Interface(gradio_fn, inputs, outputs, capture_session=True,share=True).launch(debug='True')